In [1]:
from synthesizeme.personalrm.synthesizeme import SynthesizeMe
from synthesizeme.datasets import PrismDataset

/opt/miniconda3/envs/personal_rm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Collect User Feedback

In [2]:
dataset = PrismDataset()
train, val, test = dataset.get_user_data(dataset.get_user_ids()[45])

In [3]:
data = train[:]
data.extend(val)

## View the data

In [4]:
data[0]

{'conversation_id': 'c2956',
 'user_id': 'user549',
 'context': array([{'content': 'Family value', 'role': 'user'}], dtype=object),
 'chosen': {'content': 'I can only guess, but it sounds like you are asking what family values mean to me.\n\nFamily values are important to me because they are the shared beliefs and principles that guide your behavior and interactions within a family. These values can include things like honesty, kindness, respect, forgiveness, and love.\n\nI believe that having a strong set of family values can help to create a healthy and happy home environment, where everyone feels safe, supported, and loved. It can also help to build strong relationships between family members and create a sense of belonging.\n\nFamily values are often passed down from one generation to the next, and they can be a source of comfort and stability in times of uncertainty. By teaching and modeling these values, parents can help their children to develop a strong sense of identity and bu

## Train Personal Reward Model

In [5]:
# rm = SynthesizeMe(model_id='together_ai/meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8', num_workers=64)
rm = SynthesizeMe(model_id='gemini/gemini-2.0-flash', num_workers=64)

In [6]:
rm.fit(data)

Progress:   0%|          | 0/100 [00:00<?, ?it/s]2025/05/22 02:17:43 WARNING dspy.primitives.module: There is a mismatch of python version between saved model and current environment. You saved with `python==3.12`, but now you have `python==3.10`. This might cause errors or performance downgrade on the loaded model, please consider loading the model in the same environment as the saving environment.
2025/05/22 02:17:43 WARNING dspy.primitives.module: There is a mismatch of dspy version between saved model and current environment. You saved with `dspy==2.6.12`, but now you have `dspy==2.6.24`. This might cause errors or performance downgrade on the loaded model, please consider loading the model in the same environment as the saving environment.
Tested 13 potential demo sets with persona.: 100%|██████████| 100.0/100 [00:00<00:00, 411.08it/s]            


judge_persona.predict = Predict(StringSignature(conversation, first_completion, second_completion -> reasoning, preference
    instructions="Given a conversation and two completions from different models, alongside some prior judgements and a user persona, determine which completion the human judge is more likely to prefer.  Use any provided context as well as the provided persona to speculate about the personal preferences of the judge.  You are a personalized reward model for this user, so think carefully about what this user will like.\nThe user you are judging completions for has the FOLLOWING PERSONA: ===\nThe synthesized persona is that of an individual who is well-informed, analytical, and seeks a comprehensive understanding of complex issues. They value factual accuracy and appreciate historical context and nuanced explanations. This person is likely interested in social and political topics and prefers balanced perspectives that acknowledge complexity and avoid oversimplificat

In [7]:
rm.get_persona()

'The synthesized persona is that of an individual who is well-informed, analytical, and seeks a comprehensive understanding of complex issues. They value factual accuracy and appreciate historical context and nuanced explanations. This person is likely interested in social and political topics and prefers balanced perspectives that acknowledge complexity and avoid oversimplification. They are not easily swayed by biased or unsubstantiated claims and prefer well-researched and thoughtfully presented information.'

In [8]:
if rm.get_demos():
    print(rm.get_demos()[0])

{'conversation': '[user]: Family value', 'first_completion': '[model]: I can only guess, but it sounds like you are asking what family values mean to me.\n\nFamily values are important to me because they are the shared beliefs and principles that guide your behavior and interactions within a family. These values can include things like honesty, kindness, respect, forgiveness, and love.\n\nI believe that having a strong set of family values can help to create a healthy and happy home environment, where everyone feels safe, supported, and loved. It can also help to build strong relationships between family members and create a sense of belonging.\n\nFamily values are often passed down from one generation to the next, and they can be a source of comfort and stability in times of uncertainty. By teaching and modeling these values, parents can help their children to develop a strong sense of identity and build confidence.\n\nIn short, family values are important to me because they are the f

In [9]:
rm.evaluate(test)

{'mean': 0.375,
 'lower_bound': 0.125,
 'upper_bound': 0.625,
 'results': [0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]}